In [170]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

In [171]:
df = pd.read_csv('../csv/Hotel_Reviews.csv')

In [172]:
df.isna().sum()

Hotel_Address                                    0
Additional_Number_of_Scoring                     0
Review_Date                                      0
Average_Score                                    0
Hotel_Name                                       0
Reviewer_Nationality                             0
Negative_Review                                  0
Review_Total_Negative_Word_Counts                0
Total_Number_of_Reviews                          0
Positive_Review                                  0
Review_Total_Positive_Word_Counts                0
Total_Number_of_Reviews_Reviewer_Has_Given       0
Reviewer_Score                                   0
Tags                                             0
days_since_review                                0
lat                                           3268
lng                                           3268
dtype: int64

In [173]:
df.dropna(inplace=True)

In [174]:
# Reducing the size of the dataframe to 20%
df = df.sample(frac=0.01, random_state=1)

In [175]:
df.shape

(5125, 17)

In [176]:
df['geom'] = df['lat'].map(str) + ',' + df['lng'].map(str)

In [178]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
def city(coord):
    location = geolocator.reverse(coord, exactly_one=True)
    address = location.raw['address']
    city = address.get('city', '')
    return city

def state(coord):
    location = geolocator.reverse(coord, exactly_one=True)
    address = location.raw['address']
    state = address.get('state', '')
    return 

In [179]:
df['city'] = df['geom'].apply(city)

In [180]:
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,geom,city
261673,92 Southampton Row Camden London WC1B 4BH Unit...,429,9/18/2016,8.3,DoubleTree by Hilton London West End,United Kingdom,N a,3,1774,Excellent staff,3,1,9.2,"[' Business trip ', ' Group ', ' Twin Room ', ...",319 day,51.520106,-0.122139,"51.5201065,-0.1221393",London
158643,35 Charles Street Mayfair Westminster Borough ...,252,9/20/2015,9.1,The Chesterfield Mayfair,United Kingdom,Some aspects of the room were looking a littl...,54,1166,The staff were friendly and helpful at all times,10,4,7.5,"[' Leisure trip ', ' Couple ', ' Classic Doubl...",683 day,51.507690,-0.147136,"51.507689899999995,-0.14713579999999998",City of Westminster
491784,Via Tarchetti 2 Milan City Center 20121 Milan ...,436,4/21/2017,8.4,NH Milano Touring,United Kingdom,The room was with twin bed,7,4568,I requested 3 room with one double bed each r...,33,1,5.8,"[' Business trip ', ' Couple ', ' Standard Dou...",104 day,45.476917,9.196665,"45.476916700000004,9.196665",Milano
88566,2 Caxton Street Westminster Borough London SW1...,231,3/29/2016,8.9,St Ermin s Hotel Autograph Collection,United Kingdom,nothing,2,857,staff very friendly bed very comfortable hote...,13,2,10.0,"[' Leisure trip ', ' Couple ', ' Queen Room ',...",492 day,51.498905,-0.133909,"51.498905,-0.13390929999999998",City of Westminster
456663,Spuistraat 288 292 Amsterdam City Center 1012 ...,407,7/10/2017,8.2,NH City Centre Amsterdam,Australia,The bathroom is quite spacious but it s not e...,25,3417,We had a spacious triple room our adult daugh...,77,8,9.2,"[' Leisure trip ', ' Couple ', ' Standard Trip...",24 days,52.369755,4.888782,"52.3697554,4.8887815",Amsterdam
